# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import joblib
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hd_run'

experiment=Experiment(ws, experiment_name)

In [ ]:
cpu_cluster_name = "compute-cluster"
vm_size = "Standard_D3_V2"
try:
    compute_target = ComputeTarget(workspace = ws, name= cpu_cluster_name)
    print("Found already existing")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=5, slack_factor=0.5)

#TODO: Create the different params that you will be using during training

ps =RandomParameterSampling( {
        "--C": uniform(0.001, 100),
        "--max_iter": choice(10, 20, 30, 40,50,60,70,80,90,100,150,200,300)
    }
)
#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory=".", compute_target=compute_target, entry_script="train.py")
hyperdrive_run_config = HyperDriveConfig(
                                      hyperparameter_sampling=ps,
                                      primary_metric_name="Accuracy", 
                                      estimator=est, 
                                      policy=early_termination_policy,
                                      primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                      max_total_runs=12
                                    )

In [ ]:
#TODO: Submit your experiment
hd_run = exp.submit(hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hd_run).show()
hd_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
hd_best_run = hd_run.get_best_run_by_primary_metric()

In [ ]:
hd_best_run

In [ ]:
best_run_metrics = hd_best_run.get_metrics()
parameter_values = hd_best_run.get_details()

print("Best Run Id: ", hd_best_run.id)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

In [ ]:
hd_best_model = hd_best_run.register_model(model_name="hd_model" , model_path='outputs')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service